## Libraries

In [38]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import json

In [39]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## Script

In [40]:
products=[]

n=0
pages=1
print(f"Scraped Page No-",end=' ')
while True:
  try:
    url=(f'https://www.kohls.com/catalog/sale.jsp?CN=Promotions:Sale&cc=saleclearance-TN1.0-S-saleclearance&kls_sbp=84519237627351370762922571868863011665&sks=true&PPP=48&WS={n}&S=1')
    
    page=urlopen(url).read()

    clean=BeautifulSoup(page,"html.parser")

    productlist=clean.find_all("ul",{"class":"products"})
    for content in productlist:
      for prod1 in content.find_all("li",{"class":"products_grid "}):
        products.append(prod1)
      for prod2 in content.find_all("li",{"class":"products_grid yourPrice_eligible"}):
        products.append(prod2)

    n+=48
    print(f'{pages},',end='')
    pages+=1
    
    ##comment if cell for all pages
    if pages == 100:
      break
  except:
    break


Scraped Page No- 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,

In [41]:
len(products)

3508

In [42]:
data=[]
n=0
for product in products:
  try:
    image=product.find_all('img')[0]['data-herosrc']
    name=product.find_all('p')[0].text.strip()
    try:
      sale_price=product.find_all('span',{'class':'prod_price_amount red_color'})[0].text
    except:
      sale_price=product.find_all('span',{'class':'prod_price_amount'})[0].text
    try:
      original_price=product.find_all('div',{'class':'prod_price_original'})[0].text.split()[1]
    except:
      original_price=product.find_all('div',{'class':'prod_price_original'})[0].text.split()[0]
    link='https://www.kohls.com/' + product.find_all('a')[0]['href']
    
    if not sale_price:
      sale_price='FOR PRICE, ADD TO BAG'

    row=pd.Series({'Image':image,'Name':name,'Link':link,'Original Price':original_price,'Sale Price':sale_price})

    data.append(row)
  except:
    continue

data=pd.concat(data,axis=1).T

In [43]:
len(data)

3508

In [44]:
data['id'] = data['Image'].str.extract('(\d+)').astype(int)
columnsTitles = ['id', 'Image', 'Name', 'Link', 'Original Price', 'Sale Price']

data = data.reindex(columns=columnsTitles)

## Export

In [45]:
e_folder = 'data/'
e_file = 'kohls_sales.csv'
e_path = e_folder + e_file

data.to_csv(e_path, index = False)
    